<a href="https://colab.research.google.com/github/iampatgrady/cloud-for-marketing/blob/master/Adswerve_%7C_Cloud_Marketing_Demos_%7C_End_to_End_GTM_First_Party_Mode_on_Google_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End Tag Manager w/ First-Party Mode
**Secure digital measurement using Cloud Run, CDN, and Load Balancer**  
View solution deployment here: https://www.cloud-marketing-demos.com  
Notice the gtm.js & gtag.js assets are loaded from `<fqdn>/metrics/...`

This Colab notebook automates the deployment of a web application and the necessary Google Cloud infrastructure to enable Google Tag Manager (GTM) running in First-Party Mode, following the manual setup guide: [Google Tag Manager First-Party Setup Guide.](https://developers.google.com/tag-platform/tag-manager/first-party/setup-guide?setup=manual#google-cloud)

In [2]:
# authenticate colab
from google.colab import auth
auth.authenticate_user()

# --- Core Project & Domain Settings ---
PROJECT_ID = 'cloud-for-marketing-demo' #@param {type:'string'}
DOMAIN_NAME = 'cloud-marketing-demos.com' #@param {type:'string'}
ZONE_NAME = "dns-zone" # @param {type:"string"}


# --- Cloud Run Target ---
CLOUD_RUN_SERVICE="cloud-marketing-demos" #@param {type:'string'}
CLOUD_RUN_REGION="us-central1" #@param {type:'string'}

# --- Load Balancer Naming ---
#@markdown This prefix is used to name Load Balancer and related components.
NAMING_PREFIX="metrics-lb" #@param {type:'string'}

# --- Google Tag Manager (GTM) First-Party Serving Settings ---
#@markdown Configure settings for routing GTM traffic via the Load Balancer.
# Your Google Tag Measurement ID (e.g., G-XXXXXXXXXX or AW-XXXXXXXXXX)
GTM_TAG_ID="G-DS0F7HKZTC" #@param {type:'string'}
# The path on your domain to serve GTM scripts from (e.g., /metrics, /gtm-proxy)
GTM_TAG_PATH="/metrics" #@param {type:'string'}

# --- Verification Settings (for GTM setup) ---
#@markdown Advanced: Adjust timing for GTM setup verification checks.
GTM_VERIFY_MAX_RETRIES=15 #@param {type:"integer"}
GTM_VERIFY_WAIT_SECONDS=30 #@param {type:"integer"}

# --- Set Environment Variables ---
# --- DO NOT EDIT BELOW THIS LINE ---
import os

# Basic Config
os.environ['PROJECT_ID'] = PROJECT_ID
os.environ['DOMAIN_NAME'] = DOMAIN_NAME
os.environ['CLOUD_RUN_REGION'] = CLOUD_RUN_REGION
os.environ['CLOUD_RUN_SERVICE'] = CLOUD_RUN_SERVICE

# Load Balancer Naming
os.environ['LB_PREFIX'] = NAMING_PREFIX
os.environ['GCLOUD_NETWORK_ENDPOINT_GROUP'] = f"{NAMING_PREFIX}-neg"
os.environ['GCLOUD_BACKEND_SERVICE'] = f"{NAMING_PREFIX}-backend-service"
os.environ['GCLOUD_URL_MAP'] = f"{NAMING_PREFIX}-url-map-https"
os.environ['GCLOUD_URL_MAP_HTTP_REDIRECT'] = f"{NAMING_PREFIX}-url-map-http-redirect"
os.environ['GCLOUD_SSL_CERTIFICATE'] = f"{NAMING_PREFIX}-ssl-certificate"
os.environ['GCLOUD_TARGET_HTTPS_PROXY'] = f"{NAMING_PREFIX}-target-https-proxy"
os.environ['GCLOUD_TARGET_HTTP_PROXY'] = f"{NAMING_PREFIX}-target-http-proxy"
os.environ['GCLOUD_IP_ADDRESS'] = f"{NAMING_PREFIX}-ip"
os.environ['GCLOUD_HTTPS_FORWARDING_RULE'] = f"{NAMING_PREFIX}-forwarding-rule-https"
os.environ['GCLOUD_HTTP_FORWARDING_RULE'] = f"{NAMING_PREFIX}-forwarding-rule-http"

# GTM Config
os.environ['GTM_TAG_ID'] = GTM_TAG_ID
os.environ['GTM_TAG_PATH'] = GTM_TAG_PATH
os.environ['GTM_FPS_DOMAIN'] = f"{GTM_TAG_ID}.fps.goog" # Google's First-Party Serving domain

# GTM Derived Resource Names (Using a consistent sub-prefix)
_GTM_NAMING_PREFIX = f"{NAMING_PREFIX}-gtm"
os.environ['GTM_NEG_NAME'] = f"{_GTM_NAMING_PREFIX}-fps-neg"
os.environ['GTM_BACKEND_SERVICE_NAME'] = f"{_GTM_NAMING_PREFIX}-fps-backend-service"
os.environ['GTM_PATH_MATCHER_NAME'] = f"{_GTM_NAMING_PREFIX}-path-matcher"

# GTM Verification Config
os.environ['MAX_RETRIES'] = str(GTM_VERIFY_MAX_RETRIES)
os.environ['WAIT_SECONDS'] = str(GTM_VERIFY_WAIT_SECONDS)

print("--- Environment Variables Set ---")
print(f"PROJECT_ID: {os.environ['PROJECT_ID']}")
print(f"DOMAIN_NAME: {os.environ['DOMAIN_NAME']}")
print(f"LB_PREFIX: {os.environ['LB_PREFIX']}")
print(f"GCLOUD_URL_MAP: {os.environ['GCLOUD_URL_MAP']}")
print(f"GCLOUD_BACKEND_SERVICE: {os.environ['GCLOUD_BACKEND_SERVICE']}")
print(f"GTM_TAG_ID: {os.environ['GTM_TAG_ID']}")
print(f"GTM_TAG_PATH: {os.environ['GTM_TAG_PATH']}")
print(f"GTM_FPS_DOMAIN: {os.environ['GTM_FPS_DOMAIN']}")
print(f"GTM_NEG_NAME: {os.environ['GTM_NEG_NAME']}")
print(f"GTM_BACKEND_SERVICE_NAME: {os.environ['GTM_BACKEND_SERVICE_NAME']}")
print(f"GTM_PATH_MATCHER_NAME: {os.environ['GTM_PATH_MATCHER_NAME']}")
print(f"MAX_RETRIES: {os.environ['MAX_RETRIES']}")
print(f"WAIT_SECONDS: {os.environ['WAIT_SECONDS']}")
print("---------------------------------")

# Set gcloud project context
!gcloud config set project $PROJECT_ID

--- Environment Variables Set ---
PROJECT_ID: cloud-for-marketing-demo
DOMAIN_NAME: cloud-marketing-demos.com
LB_PREFIX: metrics-lb
GCLOUD_URL_MAP: metrics-lb-url-map-https
GCLOUD_BACKEND_SERVICE: metrics-lb-backend-service
GTM_TAG_ID: G-DS0F7HKZTC
GTM_TAG_PATH: /metrics
GTM_FPS_DOMAIN: G-DS0F7HKZTC.fps.goog
GTM_NEG_NAME: metrics-lb-gtm-fps-neg
GTM_BACKEND_SERVICE_NAME: metrics-lb-gtm-fps-backend-service
GTM_PATH_MATCHER_NAME: metrics-lb-gtm-path-matcher
MAX_RETRIES: 15
WAIT_SECONDS: 30
---------------------------------
Updated property [core/project].


## Deploy Web App
Create a simple Flask web app and deploy to Cloud Run

### Create App Files
* Creates essential files for a simple Flask web application:  
  * `requirements.txt`: Defines Python dependencies (Flask, Gunicorn).  
  * `templates/index.html`: A basic HTML page containing the GTM container snippet.  
    * **Crucially, the GTM snippet's `src` attribute is modified to point to a relative path (`/metrics/?id=GTM-XXXX`) instead of `googletagmanager.com`.** This path will be handled by the load balancer later.  
  * `main.py`: The Flask application code to serve the `index.html`.  
  * `Dockerfile`: Instructions to containerize the Flask application.  
  * `.gcloudignore`: Specifies files/directories to exclude during deployment.

In [ ]:
!rm -rf .config/
!rm -rf .ipynb_checkpoints/
!rm -rf sample_data/
!mkdir templates

In [84]:
%%writefile requirements.txt
# requirements.txt
Flask==3.1.0  # Or a newer compatible version
gunicorn==23.0.0 # For App Engine deployment

Overwriting requirements.txt


In [85]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Adswerve | GTM Server Demo</title>

    <!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
    new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
    j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
    '/metrics/?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-M83JPDDR');</script>
    <!-- End Google Tag Manager -->

</head>
<body>
    <!-- Google Tag Manager (noscript) -->
<noscript><iframe src="/metrics/"
    height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript>
    <!-- End Google Tag Manager (noscript) -->

    <h1>Welcome to the GTM Server Demo!</h1>
    <p>
       This application is to demonstrate a Tag Manager Server deployed on Cloud Run.
    </p>
    <p>
        Check the page source and your browser's network requests (filtering for 'gtm.js')
        to see the script loaded from your GTM server URL, not googletagmanager.com.
    </p>
    <button onclick="dataLayer.push({event: 'button_click', button_text: 'Test Event'});">
        Send Test Event
    </button>

</body>
</html>

Overwriting templates/index.html


In [86]:
%%writefile main.py
# main.py
import os
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    """Renders the home page."""
    # We'll pass the GTM Server URL to the template later if needed,
    # but for basic script installation, it's directly in the HTML.
    return render_template('index.html')

# This is used for local testing, App Engine uses gunicorn specified in app.yaml
if __name__ == '__main__':
    # Use port 8080 for local development, matching App Engine's default
    app.run(host='127.0.0.1', port=int(os.environ.get('PORT', 8080)), debug=True)

Overwriting main.py


In [87]:
%%writefile Dockerfile
# Use an official lightweight Python image.
# Make sure the version matches or is compatible with your app's needs (python3.11)
FROM python:3.11-slim

# Set the working directory in the container
WORKDIR /app

# Set environment variables
# Prevents Python from buffering stdout and stderr
ENV PYTHONUNBUFFERED True
# The $PORT environment variable is provided by Cloud Run. Gunicorn will use it.
# We default to 8080 for local testing convenience, Cloud Run overrides this.
ENV PORT 8080

# Copy the requirements file into the container
COPY requirements.txt .

# Install dependencies
# --no-cache-dir reduces image size
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application code into the container
# Ensure your index.html is inside a 'templates' subdirectory
COPY . .

# Command to run the application using Gunicorn
# Use exec to replace the shell process with gunicorn, allowing it to receive signals correctly
# Bind to 0.0.0.0 to accept connections from outside the container
# Use the $PORT variable provided by Cloud Run
CMD exec gunicorn --bind 0.0.0.0:$PORT main:app

Overwriting Dockerfile


In [88]:
%%writefile .gcloudignore
.config
.ipynb_checkpoints
sample_data

Overwriting .gcloudignore


### Deploy Cloud Run app
* Authenticates the Colab user to Google Cloud.  
* Uses `gcloud run deploy` to build the container image from the source code and deploy it as a managed Cloud Run service (`cloud-marketing-demos`).  
* Configures the service to allow unauthenticated public access.

In [89]:
%%shell
# Deploy using the source code in the current directory (.)
gcloud run deploy ${CLOUD_RUN_SERVICE} \
  --source . \
  --project ${PROJECT_ID} \
  --region ${CLOUD_RUN_REGION} \
  --platform managed \
  --allow-unauthenticated # Make the service publicly accessible
# Follow the prompts. It will ask for confirmation.

Building using Dockerfile and deploying container to Cloud Run service [cloud-marketing-demos] in project [cloud-for-marketing-demo] region [us-central1]
Creating temporary archive of 5 file(s) totalling 3.1 KiB before compression.
Uploading zipfile of [.] to [gs://run-sources-cloud-for-marketing-demo-us-central1/services/cloud-marketing-demos/1743189394.098728-7537332ee0f54208a86478ac43ced1e0.zip]
Service [cloud-marketing-demos] revision [cloud-marketing-demos-00009-hld] has been deployed and is serving 100 percent of traffic.
Service URL: https://cloud-marketing-demos-349473679364.us-central1.run.app


## Setup load balancer
* **Creates the core components for an External HTTPS Load Balancer:**  
  * **Serverless Network Endpoint Group (NEG):** Points to the deployed Cloud Run service.  
  * **Backend Service (Web App):** Uses the Serverless NEG as its backend. CDN can be optionally enabled here.  
  * **URL Map (HTTPS):** The primary map for HTTPS traffic, initially configured to route all traffic (`defaultService`) to the web app's Backend Service.  
  * **URL Map (HTTP Redirect):** A separate map configured to redirect all HTTP requests to HTTPS.  
  * **(Optional) SSL Certificate:** Creates a Google-managed SSL certificate for the specified domain (if not already existing/managed elsewhere).  
  * **Target HTTPS Proxy:** Uses the HTTPS URL Map and SSL Certificate.  
  * **Target HTTP Proxy:** Uses the HTTP Redirect URL Map.  
  * **(Optional) Global Static IP Address:** Reserves a static IP for the load balancer (if not already existing/managed elsewhere).  
  * **HTTPS Forwarding Rule:** Directs traffic on port 443 from the static IP to the Target HTTPS Proxy.  
  * **HTTP Forwarding Rule:** Directs traffic on port 80 from the static IP to the Target HTTP Proxy (enabling the redirect).

In [123]:
#@markdown run the cell to configure load balancer

#@markdown comment out steps 6,9 to skip creating certificate, static IP
%%shell
#!/bin/bash

# Exit script immediately if any command fails
set -e

# --- Configuration ---

# Optional: CDN - Set to true to enable, false to disable
export ENABLE_CDN="true"
# --- End Configuration ---

# --- Script Start ---
echo "--- Starting Load Balancer Setup for Cloud Run Service: ${CLOUD_RUN_SERVICE} ---"
echo "Project ID:          ${PROJECT_ID}"
echo "Cloud Run Service:   ${CLOUD_RUN_SERVICE}"
echo "Cloud Run Region:    ${CLOUD_RUN_REGION}"
echo "Domain Name:         ${DOMAIN_NAME}"
echo "Load Balancer Prefix:${LB_PREFIX}"
echo "-----------------------------------------------------"

# --- Pre-check / Cleanup ---
echo "NOTE: If errors occur about resources already existing, please run the UNINSTALL section first or ensure variable names match existing resources."
echo "-----------------------------------------------------"

# 1. Create Serverless Network Endpoint Group (NEG) for Cloud Run
echo "Creating Serverless NEG: ${GCLOUD_NETWORK_ENDPOINT_GROUP} in region ${CLOUD_RUN_REGION}..."
gcloud compute network-endpoint-groups create "${GCLOUD_NETWORK_ENDPOINT_GROUP}" \
    --project="${PROJECT_ID}" \
    --region="${CLOUD_RUN_REGION}" \
    --network-endpoint-type="serverless" \
    --cloud-run-service="${CLOUD_RUN_SERVICE}" || echo "INFO: Serverless NEG ${GCLOUD_NETWORK_ENDPOINT_GROUP} might already exist."
echo "Serverless NEG step complete."
echo "-----------------------------------------------------"

# 2. Create Backend Service
echo "Creating Backend Service: ${GCLOUD_BACKEND_SERVICE}..."
CDN_FLAG=""
if [[ "$ENABLE_CDN" == "true" ]]; then
  CDN_FLAG="--enable-cdn"
fi
gcloud compute backend-services create "${GCLOUD_BACKEND_SERVICE}" \
    --project="${PROJECT_ID}" \
    --load-balancing-scheme=EXTERNAL_MANAGED \
    --global \
    $CDN_FLAG || echo "INFO: Backend Service ${GCLOUD_BACKEND_SERVICE} might already exist."
echo "Backend Service step complete."
echo "-----------------------------------------------------"

# 3. Add NEG to Backend Service
echo "Adding NEG ${GCLOUD_NETWORK_ENDPOINT_GROUP} to Backend Service ${GCLOUD_BACKEND_SERVICE}..."
# Check if backend service exists before adding backend
if gcloud compute backend-services describe "${GCLOUD_BACKEND_SERVICE}" --project="${PROJECT_ID}" --global &> /dev/null; then
    # Check if the backend is already added - Handle potential errors if it is
    if ! gcloud compute backend-services describe "${GCLOUD_BACKEND_SERVICE}" --project="${PROJECT_ID}" --global --format='value(backends.group)' | grep -q "${GCLOUD_NETWORK_ENDPOINT_GROUP}"; then
        gcloud compute backend-services add-backend "${GCLOUD_BACKEND_SERVICE}" \
            --project="${PROJECT_ID}" \
            --global \
            --network-endpoint-group="${GCLOUD_NETWORK_ENDPOINT_GROUP}" \
            --network-endpoint-group-region="${CLOUD_RUN_REGION}"
        echo "NEG added to Backend Service."
    else
        echo "INFO: NEG ${GCLOUD_NETWORK_ENDPOINT_GROUP} already added to Backend Service ${GCLOUD_BACKEND_SERVICE}."
    fi
else
    echo "ERROR: Backend service ${GCLOUD_BACKEND_SERVICE} not found. Cannot add NEG. Exiting."
    exit 1
fi
echo "-----------------------------------------------------"

# 4. Create URL Map for HTTPS traffic (pointing to the backend service)
echo "Creating HTTPS URL Map: ${GCLOUD_URL_MAP}..."
gcloud compute url-maps create "${GCLOUD_URL_MAP}" \
    --project="${PROJECT_ID}" \
    --default-service "${GCLOUD_BACKEND_SERVICE}" \
    --global || echo "INFO: HTTPS URL Map ${GCLOUD_URL_MAP} might already exist."
echo "HTTPS URL Map step complete."
echo "-----------------------------------------------------"

# 5. Create URL Map for HTTP -> HTTPS Redirect using YAML import
echo "Creating URL Map for HTTP->HTTPS Redirect: ${GCLOUD_URL_MAP_HTTP_REDIRECT} via YAML import..."
# Define the YAML content for the redirect
HTTP_REDIRECT_YAML_FILE="http_redirect_map.yaml"
cat << EOF > ${HTTP_REDIRECT_YAML_FILE}
name: ${GCLOUD_URL_MAP_HTTP_REDIRECT}
defaultUrlRedirect:
  httpsRedirect: true
  redirectResponseCode: MOVED_PERMANENTLY_DEFAULT
  stripQuery: false # Keep query parameters during redirect
EOF

# Import the URL map from the YAML file
# Use 'import' which creates if not exists, or 'update' if you expect it to exist
# Using 'import' is generally safer here if running the script multiple times
gcloud compute url-maps import "${GCLOUD_URL_MAP_HTTP_REDIRECT}" \
    --project="${PROJECT_ID}" \
    --source="${HTTP_REDIRECT_YAML_FILE}" \
    --global || echo "WARNING: Failed to import/create HTTP Redirect URL Map ${GCLOUD_URL_MAP_HTTP_REDIRECT}. It might already exist with incompatible settings, or there was a YAML/permissions issue."

# Clean up the temporary YAML file
rm -f ${HTTP_REDIRECT_YAML_FILE}
echo "HTTP Redirect URL Map configuration step complete." # Command might have issued a warning above if it failed.
echo "-----------------------------------------------------"


# 6. Create Google-Managed SSL Certificate (UNCOMMENT IF NEEDED)
echo "Creating Google-Managed SSL Certificate: ${GCLOUD_SSL_CERTIFICATE} for domain ${DOMAIN_NAME}..."
gcloud compute ssl-certificates create "${GCLOUD_SSL_CERTIFICATE}" \
    --project="${PROJECT_ID}" \
    --domains="${DOMAIN_NAME}" \
    --global || echo "INFO: SSL Certificate ${GCLOUD_SSL_CERTIFICATE} might already exist."
echo "SSL Certificate creation initiated (or already exists). Provisioning may take time."
echo "Use 'gcloud compute ssl-certificates describe ${GCLOUD_SSL_CERTIFICATE} --global --format=\"value(managed.status)\"' to check status."
echo "-----------------------------------------------------"

# 7. Create Target HTTPS Proxy
echo "Creating Target HTTPS Proxy: ${GCLOUD_TARGET_HTTPS_PROXY}..."
# Ensure the SSL cert exists before creating the proxy - crucial if step 6 is commented
if gcloud compute ssl-certificates describe "${GCLOUD_SSL_CERTIFICATE}" --project="${PROJECT_ID}" --global &> /dev/null; then
    gcloud compute target-https-proxies create "${GCLOUD_TARGET_HTTPS_PROXY}" \
        --project="${PROJECT_ID}" \
        --url-map="${GCLOUD_URL_MAP}" \
        --ssl-certificates="${GCLOUD_SSL_CERTIFICATE}" \
        --global || echo "INFO: Target HTTPS Proxy ${GCLOUD_TARGET_HTTPS_PROXY} might already exist."
    echo "Target HTTPS Proxy step complete."
else
    echo "ERROR: SSL Certificate ${GCLOUD_SSL_CERTIFICATE} not found. Cannot create Target HTTPS Proxy. Exiting."
    echo "If you commented out step 6, ensure the certificate name is correct and the certificate exists."
    exit 1
fi
echo "-----------------------------------------------------"

# 8. Create Target HTTP Proxy (for the redirect)
echo "Creating Target HTTP Proxy: ${GCLOUD_TARGET_HTTP_PROXY}..."
# Check if the redirect URL map exists before creating the proxy
if gcloud compute url-maps describe "${GCLOUD_URL_MAP_HTTP_REDIRECT}" --project="${PROJECT_ID}" --global &> /dev/null; then
    gcloud compute target-http-proxies create "${GCLOUD_TARGET_HTTP_PROXY}" \
        --project="${PROJECT_ID}" \
        --url-map="${GCLOUD_URL_MAP_HTTP_REDIRECT}" \
        --global || echo "INFO: Target HTTP Proxy ${GCLOUD_TARGET_HTTP_PROXY} might already exist."
    echo "Target HTTP Proxy step complete."
else
    echo "WARNING: HTTP Redirect URL Map (${GCLOUD_URL_MAP_HTTP_REDIRECT}) not found (Import in Step 5 likely failed). Skipping Target HTTP Proxy creation. HTTP->HTTPS redirect will not function."
fi
echo "-----------------------------------------------------"

# 9. Reserve Global Static IP Address (UNCOMMENT IF NEEDED)
echo "Reserving Global Static IP Address: ${GCLOUD_IP_ADDRESS}..."
gcloud compute addresses create "${GCLOUD_IP_ADDRESS}" \
    --project="${PROJECT_ID}" \
    --ip-version=IPV4 \
    --network-tier=PREMIUM \
    --global || echo "INFO: IP Address ${GCLOUD_IP_ADDRESS} might already exist."
echo "IP Address reservation step complete."
echo "-----------------------------------------------------"

# Get the reserved IP address value - Crucial step even if Step 9 is commented
echo "Fetching IP Address: ${GCLOUD_IP_ADDRESS}..."
RESERVED_IP=""
# Try fetching the IP a few times - it must exist
for i in {1..5}; do
  RESERVED_IP=$(gcloud compute addresses describe "${GCLOUD_IP_ADDRESS}" --project="${PROJECT_ID}" --global --format='value(address)' 2>/dev/null)
  if [[ -n "$RESERVED_IP" ]]; then
    break
  fi
  echo "Waiting for IP address ${GCLOUD_IP_ADDRESS} to be available (attempt $i)..."
  sleep 5
done

if [[ -z "$RESERVED_IP" ]]; then
  echo "ERROR: Failed to fetch IP address for ${GCLOUD_IP_ADDRESS}. Ensure the address exists and the name is correct."
  echo "If you commented out step 9, ensure the IP name is correct and the IP exists."
  exit 1
fi
echo "Load Balancer IP Address is: ${RESERVED_IP}"
echo "-----------------------------------------------------"

# 10. Create HTTPS Forwarding Rule
echo "Creating HTTPS Forwarding Rule: ${GCLOUD_HTTPS_FORWARDING_RULE}..."
gcloud compute forwarding-rules create "${GCLOUD_HTTPS_FORWARDING_RULE}" \
    --project="${PROJECT_ID}" \
    --load-balancing-scheme=EXTERNAL_MANAGED \
    --network-tier=PREMIUM \
    --address="${GCLOUD_IP_ADDRESS}" \
    --target-https-proxy="${GCLOUD_TARGET_HTTPS_PROXY}" \
    --ports=443 \
    --global || echo "INFO: HTTPS Forwarding Rule ${GCLOUD_HTTPS_FORWARDING_RULE} might already exist."
echo "HTTPS Forwarding Rule step complete."
echo "-----------------------------------------------------"

# 11. Create HTTP Forwarding Rule (for the redirect)
echo "Creating HTTP Forwarding Rule: ${GCLOUD_HTTP_FORWARDING_RULE}..."
# Check if the target HTTP proxy exists before creating the rule
if gcloud compute target-http-proxies describe "${GCLOUD_TARGET_HTTP_PROXY}" --project="${PROJECT_ID}" --global &> /dev/null; then
    gcloud compute forwarding-rules create "${GCLOUD_HTTP_FORWARDING_RULE}" \
        --project="${PROJECT_ID}" \
        --load-balancing-scheme=EXTERNAL_MANAGED \
        --network-tier=PREMIUM \
        --address="${GCLOUD_IP_ADDRESS}" \
        --target-http-proxy="${GCLOUD_TARGET_HTTP_PROXY}" \
        --ports=80 \
        --global || echo "INFO: HTTP Forwarding Rule ${GCLOUD_HTTP_FORWARDING_RULE} might already exist."
    echo "HTTP Forwarding Rule step complete."
else
     echo "WARNING: Target HTTP Proxy (${GCLOUD_TARGET_HTTP_PROXY}) not found (creation in Step 8 likely skipped). Skipping HTTP Forwarding rule creation. HTTP->HTTPS redirect will not function."
fi
echo "-----------------------------------------------------"


echo "--- Load Balancer Setup Script Finished ---"
echo ""
echo "Next Steps:"
echo "1. REVIEW SCRIPT OUTPUT CAREFULLY for any 'ERROR' or 'WARNING' messages."
echo "   'INFO' messages about existing resources are usually okay if intended."
echo "2. If resources already existed or steps failed unexpectedly, consider running an uninstall/cleanup script and trying again."
echo "3. ***IMPORTANT DNS STEP***: Update the DNS 'A' record for '${DOMAIN_NAME}' to point to the Load Balancer's IP address: ${RESERVED_IP}"
echo "   >>> This IP address (${RESERVED_IP}) is for the Load Balancer. Do NOT use the IPs shown in the Cloud Run 'Domain mappings' screen <<<"
echo "4. Wait for the SSL certificate '${GCLOUD_SSL_CERTIFICATE}' status to become ACTIVE. This can take 30-90 minutes *after* DNS propagation is complete."
echo "   Check status with: gcloud compute ssl-certificates describe ${GCLOUD_SSL_CERTIFICATE} --global --format='value(managed.status)'"
echo "5. If the HTTP redirect steps (5, 8, 11) showed WARNINGS, HTTP traffic (port 80) will NOT automatically redirect to HTTPS (port 443). Your site will only be reliably accessible via 'https://${DOMAIN_NAME}'."
echo ""
echo "Load Balancer IP Address: ${RESERVED_IP}"
echo "-----------------------------------------------------"

--- Starting Load Balancer Setup for Cloud Run Service: cloud-marketing-demos ---
Project ID:          cloud-for-marketing-demo
Cloud Run Service:   cloud-marketing-demos
Cloud Run Region:    us-central1
Domain Name:         cloud-marketing-demos.com
Load Balancer Prefix:metrics-lb
-----------------------------------------------------
NOTE: If errors occur about resources already existing, please run the UNINSTALL section first or ensure variable names match existing resources.
-----------------------------------------------------
Creating Serverless NEG: metrics-lb-neg in region us-central1...
Created [https://www.googleapis.com/compute/v1/projects/cloud-for-marketing-demo/regions/us-central1/networkEndpointGroups/metrics-lb-neg].
Created network endpoint group [metrics-lb-neg].
Serverless NEG step complete.
-----------------------------------------------------
Creating Backend Service: metrics-lb-backend-service...
Created [https://www.googleapis.com/compute/v1/projects/cloud-for-mar

### uninstall - load balancer

In [122]:
#@markdown uncomment and run to restore cloud project to previous state

#@markdown uncomment`# --- OPTIONAL DELETIONS ---` (certificates, IP) if needed

# %%shell
# #!/bin/bash

# # Allow script to continue even if a command fails, reporting errors
# set +e



# echo "--- Starting Load Balancer Resource Deletion ---"
# echo "Project ID:          ${PROJECT_ID}"
# echo "Cloud Run Region:    ${CLOUD_RUN_REGION}"
# echo "Load Balancer Prefix:${LB_PREFIX}"
# echo "NOTE: This script will attempt to delete resources. It's okay if some commands report 'Not Found' if they were already deleted or failed to create initially."
# echo "NOTE: Using '--quiet' to avoid interactive prompts."
# echo "-----------------------------------------------------"

# # 1. Delete HTTP Forwarding Rule
# echo "Deleting HTTP Forwarding Rule: ${GCLOUD_HTTP_FORWARDING_RULE}..."
# gcloud compute forwarding-rules delete "${GCLOUD_HTTP_FORWARDING_RULE}" \
#     --project="${PROJECT_ID}" \
#     --global \
#     --quiet || echo "WARNING: Failed to delete HTTP Forwarding Rule ${GCLOUD_HTTP_FORWARDING_RULE} (may not exist)."
# echo "-----------------------------------------------------"

# # 2. Delete HTTPS Forwarding Rule
# echo "Deleting HTTPS Forwarding Rule: ${GCLOUD_HTTPS_FORWARDING_RULE}..."
# gcloud compute forwarding-rules delete "${GCLOUD_HTTPS_FORWARDING_RULE}" \
#     --project="${PROJECT_ID}" \
#     --global \
#     --quiet || echo "WARNING: Failed to delete HTTPS Forwarding Rule ${GCLOUD_HTTPS_FORWARDING_RULE} (may not exist)."
# echo "-----------------------------------------------------"

# # 3. Delete Target HTTP Proxy
# echo "Deleting Target HTTP Proxy: ${GCLOUD_TARGET_HTTP_PROXY}..."
# gcloud compute target-http-proxies delete "${GCLOUD_TARGET_HTTP_PROXY}" \
#     --project="${PROJECT_ID}" \
#     --global \
#     --quiet || echo "WARNING: Failed to delete Target HTTP Proxy ${GCLOUD_TARGET_HTTP_PROXY} (may not exist)."
# echo "-----------------------------------------------------"

# # 4. Delete Target HTTPS Proxy
# echo "Deleting Target HTTPS Proxy: ${GCLOUD_TARGET_HTTPS_PROXY}..."
# gcloud compute target-https-proxies delete "${GCLOUD_TARGET_HTTPS_PROXY}" \
#     --project="${PROJECT_ID}" \
#     --global \
#     --quiet || echo "WARNING: Failed to delete Target HTTPS Proxy ${GCLOUD_TARGET_HTTPS_PROXY} (may not exist)."
# echo "-----------------------------------------------------"

# # 5. Delete URL Map for HTTP Redirect
# echo "Deleting HTTP Redirect URL Map: ${GCLOUD_URL_MAP_HTTP_REDIRECT}..."
# gcloud compute url-maps delete "${GCLOUD_URL_MAP_HTTP_REDIRECT}" \
#     --project="${PROJECT_ID}" \
#     --global \
#     --quiet || echo "WARNING: Failed to delete HTTP Redirect URL Map ${GCLOUD_URL_MAP_HTTP_REDIRECT} (may not exist)."
# echo "-----------------------------------------------------"

# # 6. Delete URL Map for HTTPS
# echo "Deleting HTTPS URL Map: ${GCLOUD_URL_MAP}..."
# gcloud compute url-maps delete "${GCLOUD_URL_MAP}" \
#     --project="${PROJECT_ID}" \
#     --global \
#     --quiet || echo "WARNING: Failed to delete HTTPS URL Map ${GCLOUD_URL_MAP} (may not exist)."
# echo "-----------------------------------------------------"

# # 7. Remove NEG from Backend Service (Must happen before deleting Backend Service)
# echo "Checking if Backend Service ${GCLOUD_BACKEND_SERVICE} exists before removing NEG..."
# if gcloud compute backend-services describe "${GCLOUD_BACKEND_SERVICE}" --project="${PROJECT_ID}" --global &> /dev/null; then
#     echo "Removing NEG ${GCLOUD_NETWORK_ENDPOINT_GROUP} from Backend Service ${GCLOUD_BACKEND_SERVICE}..."
#     gcloud compute backend-services remove-backend "${GCLOUD_BACKEND_SERVICE}" \
#         --project="${PROJECT_ID}" \
#         --global \
#         --network-endpoint-group="${GCLOUD_NETWORK_ENDPOINT_GROUP}" \
#         --network-endpoint-group-region="${CLOUD_RUN_REGION}" \
#         --quiet || echo "WARNING: Failed to remove NEG ${GCLOUD_NETWORK_ENDPOINT_GROUP} from Backend Service ${GCLOUD_BACKEND_SERVICE} (may have already been removed)."
# else
#     echo "INFO: Backend Service ${GCLOUD_BACKEND_SERVICE} not found, skipping NEG removal."
# fi
# echo "-----------------------------------------------------"

# # 8. Delete Backend Service
# echo "Deleting Backend Service: ${GCLOUD_BACKEND_SERVICE}..."
# gcloud compute backend-services delete "${GCLOUD_BACKEND_SERVICE}" \
#     --project="${PROJECT_ID}" \
#     --global \
#     --quiet || echo "WARNING: Failed to delete Backend Service ${GCLOUD_BACKEND_SERVICE} (may not exist or still have dependencies)."
# echo "-----------------------------------------------------"

# # 9. Delete Serverless Network Endpoint Group (NEG)
# echo "Deleting Serverless NEG: ${GCLOUD_NETWORK_ENDPOINT_GROUP} in region ${CLOUD_RUN_REGION}..."
# gcloud compute network-endpoint-groups delete "${GCLOUD_NETWORK_ENDPOINT_GROUP}" \
#     --project="${PROJECT_ID}" \
#     --region="${CLOUD_RUN_REGION}" \
#     --quiet || echo "WARNING: Failed to delete Serverless NEG ${GCLOUD_NETWORK_ENDPOINT_GROUP} (may not exist)."
# echo "-----------------------------------------------------"

# # --- OPTIONAL DELETIONS ---
# # Uncomment the sections below if you also want to delete the
# # SSL Certificate and the Reserved IP Address.
# # WARNING: Deleting certificates/IPs can impact other services if they are shared.
# #          Only uncomment if you are sure they are dedicated to this LB setup.

# # # 10. OPTIONAL: Delete SSL Certificate
# # echo "Attempting OPTIONAL delete of SSL Certificate: ${GCLOUD_SSL_CERTIFICATE}..."
# # # NOTE: Certificate deletion might fail if it's still attached to a resource (e.g., proxy not fully deleted yet)
# # #       or if it's managed by another process. May need to retry later.
# # gcloud compute ssl-certificates delete "${GCLOUD_SSL_CERTIFICATE}" \
# #     --project="${PROJECT_ID}" \
# #     --global \
# #     --quiet || echo "WARNING: Failed to delete SSL Certificate ${GCLOUD_SSL_CERTIFICATE} (may not exist or still be in use)."
# # echo "-----------------------------------------------------"

# # # 11. OPTIONAL: Delete Reserved IP Address
# # echo "Attempting OPTIONAL delete of Reserved IP Address: ${GCLOUD_IP_ADDRESS}..."
# # # NOTE: IP Address deletion might fail if it's still attached to a resource (e.g., forwarding rule not fully deleted yet).
# # #       May need to retry later.
# # gcloud compute addresses delete "${GCLOUD_IP_ADDRESS}" \
# #     --project="${PROJECT_ID}" \
# #     --global \
# #     --quiet || echo "WARNING: Failed to delete IP Address ${GCLOUD_IP_ADDRESS} (may not exist or still be in use)."
# # echo "-----------------------------------------------------"

# echo "--- Load Balancer Resource Deletion Script Finished ---"
# echo "Review the output above for any WARNING messages."
# echo "If warnings occurred (especially for optional steps or dependencies), you may need to manually check resource status in the Cloud Console or retry failed commands after waiting a few minutes."
# echo "-----------------------------------------------------"

## DNS & Certs
* Includes commands to check if the DNS 'A' record for the domain points to the load balancer's static IP (`dig`).  
* Provides a Python loop to poll the status of the Google-managed SSL certificate until it becomes `ACTIVE` (this requires DNS to be correctly pointing to the LB IP).  
* Includes a Python snippet to make an HTTP GET request to the domain to verify the site is live and accessible via HTTPS after the certificate provisions.

In [3]:
# ==============================================================================
# PART 1: CREATE/UPDATE DNS 'A' RECORDS IN GOOGLE CLOUD DNS
# ==============================================================================
import subprocess
import os
import sys
import time
from IPython.display import clear_output

# @markdown ---
# @markdown ### Google Cloud DNS Configuration:
# @markdown ---

# Ensure required environment variables are set (assuming these are set elsewhere in your Colab)
required_env_vars = ['PROJECT_ID', 'GCLOUD_IP_ADDRESS']
missing_vars = [var for var in required_env_vars if var not in os.environ]
if missing_vars:
    print(f"❌ ERROR: Missing required environment variables: {', '.join(missing_vars)}")
    print("Please set PROJECT_ID (your Google Cloud Project ID) and GCLOUD_IP_ADDRESS (the *name* of your Global IP Address resource).")
    # Stop execution if variables are missing
    sys.exit(1) # Or raise an exception

# Set the DOMAIN_NAME environment variable so the checking script can use it
os.environ['DOMAIN_NAME'] = DOMAIN_NAME.strip()

# --- Helper function to run gcloud commands ---
def run_gcloud_command(command, description):
    """Runs a gcloud command and handles errors."""
    print(f"🔄 {description}...")
    try:
        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            check=True, # Raises CalledProcessError on non-zero exit codes
            timeout=60 # Add a timeout
        )
        print(f"✅ {description} successful.")
        # print(f"Output:\n{result.stdout}") # Optional: print stdout on success
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"❌ ERROR: {description} failed.")
        print(f"Command: {e.cmd}")
        print(f"Return Code: {e.returncode}")
        print(f"Stderr:\n{e.stderr}")
        print(f"Stdout:\n{e.stdout}")
        # Re-raise the exception or exit if you want the script to stop on failure
        raise e # Or sys.exit(1)
    except subprocess.TimeoutExpired:
        print(f"❌ ERROR: {description} timed out.")
        raise # Or sys.exit(1)
    except Exception as e:
        print(f"❌ ERROR: An unexpected error occurred during '{description}'.")
        print(f"Error type: {type(e).__name__}")
        print(f"Error details: {e}")
        raise # Or sys.exit(1)

# --- Main DNS Update Logic ---
try:
    # 1. Get the Load Balancer IP Address
    ip_command = f"gcloud compute addresses describe {os.environ['GCLOUD_IP_ADDRESS']} --project={os.environ['PROJECT_ID']} --global --format='value(address)'"
    load_balancer_ip = run_gcloud_command(ip_command, "Retrieving Load Balancer IP Address")

    if not load_balancer_ip:
        print("❌ ERROR: Failed to retrieve Load Balancer IP address. Cannot proceed.")
        sys.exit(1)

    print(f"✅ Found Load Balancer IP: {load_balancer_ip}")

    # Ensure domain name ends with a dot for FQDN
    domain_fqdn = DOMAIN_NAME.strip()
    if not domain_fqdn.endswith('.'):
        domain_fqdn += '.'

    # Construct FQDN for www subdomain
    www_domain_fqdn = f"www.{domain_fqdn}"

    # Define TTL
    ttl = "300"

    # 2. Update/Create A record for the apex domain (@)
    apex_command = (
        f"gcloud dns --project={os.environ['PROJECT_ID']} record-sets update {domain_fqdn} "
        f"--type=\"A\" --zone=\"{ZONE_NAME}\" --rrdatas=\"{load_balancer_ip}\" --ttl=\"{ttl}\""
    )
    run_gcloud_command(apex_command, f"Updating A record for {domain_fqdn}")

    # 3. Update/Create A record for the www subdomain
    www_command = (
        f"gcloud dns --project={os.environ['PROJECT_ID']} record-sets update {www_domain_fqdn} "
        f"--type=\"A\" --zone=\"{ZONE_NAME}\" --rrdatas=\"{load_balancer_ip}\" --ttl=\"{ttl}\""
    )
    run_gcloud_command(www_command, f"Updating A record for {www_domain_fqdn}")

    print("\n✅ DNS Records update commands executed successfully.")
    print("Note: DNS propagation may take some time.")

except Exception as e:
    print(f"\n❌ An error occurred during the DNS update process: {e}")
    print("Aborting further checks.")
    # Optional: exit if you don't want the check loop to run on failure
    # sys.exit(1)

# ==============================================================================
# PART 2: CHECK DNS PROPAGATION (Your Original Script)
# ==============================================================================

print("\nStarting DNS propagation check...")
# Add a small initial delay to allow potential immediate propagation
time.sleep(10)

while True:
    try:
        clear_output(wait=True) # wait=True prevents flickering
        print("Checking DNS propagation...")

        # Get IP from DNS query
        dig_command = f"dig {os.environ['DOMAIN_NAME']} A +short"
        print(f"Running: {dig_command}")
        dig_result = subprocess.run(dig_command, shell=True, capture_output=True, text=True, check=True, timeout=10)
        dns_ip = dig_result.stdout.splitlines()[0].strip() if dig_result.stdout.splitlines() else None
        print(f"DNS ({os.environ['DOMAIN_NAME']}) resolves to: {dns_ip if dns_ip else 'Not found'}")

        # Get Expected IP from gcloud
        gcloud_ip_command = f"gcloud compute addresses describe {os.environ['GCLOUD_IP_ADDRESS']} --project={os.environ['PROJECT_ID']} --global --format='value(address)'"
        # Don't print the gcloud command every loop unless debugging
        # print(f"Running: {gcloud_ip_command}")
        gcloud_result = subprocess.run(gcloud_ip_command, shell=True, capture_output=True, text=True, check=True, timeout=20)
        expected_ip = gcloud_result.stdout.strip()
        print(f"Expected Load Balancer IP: {expected_ip}")

        # Compare IPs
        if dns_ip and dns_ip == expected_ip:
            print("\n==========================================")
            print(f"✅ SUCCESS: DNS record for {os.environ['DOMAIN_NAME']} matches the Load Balancer IP!")
            print(f"IP Address: {expected_ip}")
            print("==========================================")
            break # Exit loop on match
        else:
            print(f"\n❌ IPs do not match yet (DNS: {dns_ip}, Expected: {expected_ip}). Retrying in 30 seconds...")

    except subprocess.CalledProcessError as e:
        # Handle errors from dig or gcloud specifically if needed
        print(f"Check command failed: {e}. Retrying in 30 seconds...")
        print(f"Command: {e.cmd}")
        print(f"Stderr: {e.stderr}")
    except IndexError:
         # Handle case where dig returns no IP
        print(f"Check failed: 'dig' command did not return an IP address for {os.environ['DOMAIN_NAME']}. Retrying in 30 seconds...")
    except Exception as e:
        # Catch other potential errors (timeouts, etc.)
        print(f"An unexpected error occurred during check: {type(e).__name__} - {e}. Retrying in 30 seconds...")

    time.sleep(30) # Wait 30 seconds before next check

Checking DNS propagation...
Running: dig cloud-marketing-demos.com A +short
DNS (cloud-marketing-demos.com) resolves to: 34.49.224.75
Expected Load Balancer IP: 34.49.224.75

✅ SUCCESS: DNS record for cloud-marketing-demos.com matches the Load Balancer IP!
IP Address: 34.49.224.75


In [4]:
#@markdown Run this cell to wait on cert to provision (checks every 30 seconds)
import time
from IPython.display import clear_output

def check_ssl_certificate_status():
  """Checks the SSL certificate status and returns True if it's not 'PROVISIONING'."""
  try:
    result = subprocess.check_output(
        ['gcloud', 'compute', 'ssl-certificates', 'describe', 'metrics-lb-ssl-certificate', '--global', '--format=value(managed.status)'],
        stderr=subprocess.STDOUT,
        text=True
    ).strip()
    print(f"SSL Certificate status: {result}")
    return result != 'PROVISIONING'
  except subprocess.CalledProcessError as e:
    print(f"Error checking SSL certificate status: {e.output}")
    return False

# Loop until the SSL certificate is not 'PROVISIONING' or the cell is stopped.
while True:
  clear_output()
  if check_ssl_certificate_status():
    print("SSL Certificate is not provisioning.")
    break
  print("SSL Certificate is still provisioning. Retrying in 30 seconds...")
  time.sleep(30)


SSL Certificate status: ACTIVE
SSL Certificate is not provisioning.


In [5]:
#@markdown Check to see if the site is live:
import requests

try:
  response = requests.get("https://cloud-marketing-demos.com")
  response.raise_for_status()  # Raise an exception for bad status codes
  if response.status_code == 200:
    print("GET request to https://cloud-marketing-demos.com returned status 200.")
  else:
    print(f"GET request returned unexpected status code: {response.status_code}")
except requests.exceptions.RequestException as e:
  print(f"Error during GET request: {e}")


GET request to https://cloud-marketing-demos.com returned status 200.


## Add First-Party Model Google Tag
https://developers.google.com/tag-platform/tag-manager/first-party/setup-guide?setup=manual#google-cloud


* Defines specific configuration for GTM FPM (GTM Tag ID, desired path like `/metrics`, Google's FPM domain `G-XXXX.fps.goog`).  
* **Creates GTM-specific Load Balancer components:**  
  * **Internet Network Endpoint Group (NEG):** A *global* NEG of type `INTERNET_FQDN_PORT` pointing to Google's FPM endpoint (`<GTM_TAG_ID>.fps.goog` on port 443).  
  * **Backend Service (GTM FPM):** A *second* backend service using the Internet NEG. Configured for HTTPS, without CDN, and crucially, adds **custom request headers** (`Host`, `X-Gclb-Country`, `X-Gclb-Region`) required by Google's FPM servers.  
* **Updates the Main URL Map (HTTPS):**  
  * Uses `gcloud compute url-maps import` with a generated YAML configuration.  
  * This **modifies** the existing HTTPS URL map (`metrics-lb-url-map-https`) created in step 4\.  
  * It adds a `pathMatcher` and `pathRules` to route requests specifically matching the `GTM_PATH` (e.g., `/metrics` and `/metrics/*`) to the new **GTM FPM Backend Service**.  
  * Requests *not* matching the `GTM_PATH` continue to be routed to the original **Web App Backend Service** (as the default service for the path matcher).  
* **Verification:** Runs `curl` commands in a loop to check the `/healthy` and `/?validate_geo=healthy` endpoints under the configured `GTM_PATH` (e.g., `https://your-domain.com/metrics/healthy`). This confirms the load balancer routing and GTM backend are working correctly.

### create FPM services

In [146]:
#@markdown Create GTM FPM Cloud Services
%%shell
#!/bin/bash

# Exit script immediately if any command fails
set -e

# --- Script Start ---
# Configuration is now set via environment variables from the Python cell above.
echo "--- Starting GTM First-Party Load Balancer Configuration ---"
echo "Project ID:                 ${PROJECT_ID}"
echo "Domain Name:                ${DOMAIN_NAME}"
echo "GTM Tag ID:                 ${GTM_TAG_ID}"
echo "GTM Tag Path:               ${GTM_TAG_PATH}"
echo "Load Balancer URL Map:      ${GCLOUD_URL_MAP}"
echo "Default Backend Service:    ${GCLOUD_BACKEND_SERVICE}"
echo "GTM FPS Domain:             ${GTM_FPS_DOMAIN}"
echo "GTM NEG Name:               ${GTM_NEG_NAME}"
echo "GTM Backend Service Name:   ${GTM_BACKEND_SERVICE_NAME}"
echo "GTM Path Matcher Name:      ${GTM_PATH_MATCHER_NAME}"
echo "Verification Retries:       ${MAX_RETRIES}"
echo "Verification Wait Seconds:  ${WAIT_SECONDS}"
echo "-----------------------------------------------------"

# --- Step 1 & 2: Create GTM NEG and Backend Service ---

# 1. Create Internet Network Endpoint Group (NEG) for GTM FPS
echo "Creating/Verifying Global Internet NEG: ${GTM_NEG_NAME}..."
gcloud compute network-endpoint-groups create "${GTM_NEG_NAME}" \
    --project="${PROJECT_ID}" \
    --network-endpoint-type="INTERNET_FQDN_PORT" \
    --global \
    || echo "INFO: Internet NEG ${GTM_NEG_NAME} might already exist."

echo "Adding/Verifying endpoint ${GTM_FPS_DOMAIN}:443 to NEG ${GTM_NEG_NAME}..."
gcloud compute network-endpoint-groups update "${GTM_NEG_NAME}" \
    --project="${PROJECT_ID}" \
    --global \
    --add-endpoint="fqdn=${GTM_FPS_DOMAIN},port=443" \
    || echo "INFO: Endpoint ${GTM_FPS_DOMAIN}:443 might already be added to NEG ${GTM_NEG_NAME}."
echo "Internet NEG step complete."
echo "-----------------------------------------------------"

# 2. Create Backend Service for GTM FPS
echo "Creating/Verifying Global Backend Service: ${GTM_BACKEND_SERVICE_NAME}..."
gcloud compute backend-services create "${GTM_BACKEND_SERVICE_NAME}" \
    --project="${PROJECT_ID}" \
    --protocol=HTTPS \
    --load-balancing-scheme=EXTERNAL_MANAGED \
    --global \
    --no-enable-cdn \
    || echo "INFO: GTM Backend Service ${GTM_BACKEND_SERVICE_NAME} might already exist."

echo "Adding/Verifying NEG ${GTM_NEG_NAME} to Backend Service ${GTM_BACKEND_SERVICE_NAME}..."
# Check if backend service exists before adding backend
if gcloud compute backend-services describe "${GTM_BACKEND_SERVICE_NAME}" --project="${PROJECT_ID}" --global &> /dev/null; then
    # Check if the backend is already added - Handle potential errors if it is
    if ! gcloud compute backend-services describe "${GTM_BACKEND_SERVICE_NAME}" --project="${PROJECT_ID}" --global --format='value(backends.group)' 2>/dev/null | grep -q "${GTM_NEG_NAME}"; then
        gcloud compute backend-services add-backend "${GTM_BACKEND_SERVICE_NAME}" \
            --project="${PROJECT_ID}" \
            --network-endpoint-group="${GTM_NEG_NAME}" \
            --global-network-endpoint-group \
            --global \
            --quiet
        echo "NEG added to GTM Backend Service."
    else
        echo "INFO: NEG ${GTM_NEG_NAME} already added to GTM Backend Service ${GTM_BACKEND_SERVICE_NAME}."
    fi
else
    echo "ERROR: GTM Backend service ${GTM_BACKEND_SERVICE_NAME} not found. Cannot add NEG. Exiting."
    exit 1
fi


echo "Updating/Verifying Backend Service ${GTM_BACKEND_SERVICE_NAME} with custom headers..."
gcloud compute backend-services update "${GTM_BACKEND_SERVICE_NAME}" \
    --project="${PROJECT_ID}" \
    --custom-request-header="Host: ${GTM_FPS_DOMAIN}" \
    --custom-request-header="X-Gclb-Country: {client_region}" \
    --custom-request-header="X-Gclb-Region: {client_region_subdivision}" \
    --global \
    || echo "WARNING: Failed to update headers for GTM Backend Service ${GTM_BACKEND_SERVICE_NAME}. Check if the service exists and name is correct."
echo "GTM Backend Service step complete."
echo "-----------------------------------------------------"

# --- Step 3: Configure URL Map using YAML Import ---
echo "Configuring URL Map ${GCLOUD_URL_MAP} using YAML import..."

# Define the desired URL Map configuration in a YAML file
URL_MAP_YAML_FILE="url_map_config.yaml"

# Get full URLs for backend services - Check they exist first
DEFAULT_SERVICE_URL=$(gcloud compute backend-services describe "${GCLOUD_BACKEND_SERVICE}" --project="${PROJECT_ID}" --global --format="value(selfLink)" 2>/dev/null)
GTM_SERVICE_URL=$(gcloud compute backend-services describe "${GTM_BACKEND_SERVICE_NAME}" --project="${PROJECT_ID}" --global --format="value(selfLink)" 2>/dev/null)

# Ensure backend service URLs were fetched
if [ -z "$DEFAULT_SERVICE_URL" ]; then
  echo "ERROR: Could not retrieve full URL for the default backend service (${GCLOUD_BACKEND_SERVICE}). Ensure it exists. Exiting."
  exit 1
fi
if [ -z "$GTM_SERVICE_URL" ]; then
  echo "ERROR: Could not retrieve full URL for the GTM backend service (${GTM_BACKEND_SERVICE_NAME}). Ensure it was created successfully in Step 2. Exiting."
  exit 1
fi

cat << EOF > ${URL_MAP_YAML_FILE}
# Configuration for URL Map: ${GCLOUD_URL_MAP}
# Defines routing for ${DOMAIN_NAME}
kind: compute#urlMap
name: ${GCLOUD_URL_MAP}
defaultService: ${DEFAULT_SERVICE_URL}
hostRules:
- hosts:
  - ${DOMAIN_NAME}
# - "*.${DOMAIN_NAME}" # Optional: Uncomment to include wildcards
  pathMatcher: ${GTM_PATH_MATCHER_NAME}
# Add other hosts here if needed, pointing to appropriate path matchers
pathMatchers:
- name: ${GTM_PATH_MATCHER_NAME}
  defaultService: ${DEFAULT_SERVICE_URL} # Default for this host/matcher
  pathRules:
  - paths: # Paths that go to the GTM Backend Service
    - "${GTM_TAG_PATH}"    # e.g., /metrics
    - "${GTM_TAG_PATH}/*" # e.g., /metrics/*
    service: ${GTM_SERVICE_URL}
  # Add other specific path rules for this host/matcher here if needed
  # Example: Route /admin/* to a different backend
  # - paths:
  #   - "/admin/*"
  #   service: "URL_TO_ADMIN_BACKEND_SERVICE"
EOF

echo "Generated YAML configuration:"
cat ${URL_MAP_YAML_FILE}
echo "---"

# Import the configuration, overwriting the existing URL Map
echo "Importing YAML configuration into URL Map ${GCLOUD_URL_MAP}..."
gcloud compute url-maps import "${GCLOUD_URL_MAP}" \
    --project="${PROJECT_ID}" \
    --source="${URL_MAP_YAML_FILE}" \
    --global \
    --quiet

# Clean up the temporary YAML file
rm -f ${URL_MAP_YAML_FILE}

echo "URL Map configuration step complete."
echo "-----------------------------------------------------"


# 4. Verification with Polling Loop
echo "Starting verification for https://${DOMAIN_NAME}${GTM_TAG_PATH}/healthy..."
retries=0
healthy_ok=false
geo_ok=false
VERIFY_URL_HEALTHY="https://${DOMAIN_NAME}${GTM_TAG_PATH}/healthy"
VERIFY_URL_GEO="https://${DOMAIN_NAME}${GTM_TAG_PATH}/?validate_geo=healthy"


# Use MAX_RETRIES and WAIT_SECONDS from environment variables
while [[ $retries -lt $MAX_RETRIES ]]; do
    echo "Attempt $((retries + 1))/${MAX_RETRIES}: Checking endpoints..."

    # Check /healthy endpoint
    echo "  Checking ${VERIFY_URL_HEALTHY}..."
    HTTP_STATUS_HEALTHY=$(curl --connect-timeout 10 -s -L -o /dev/null -w "%{http_code}" "${VERIFY_URL_HEALTHY}" || echo "curl_error")
    if [[ "$HTTP_STATUS_HEALTHY" == "200" ]]; then
        HEALTHY_OUTPUT=$(curl --connect-timeout 10 -s -L "${VERIFY_URL_HEALTHY}" || echo "curl_error")
        if [[ "$HEALTHY_OUTPUT" == "ok" ]]; then
            echo "  /healthy check: OK (Status: ${HTTP_STATUS_HEALTHY}, Body: ${HEALTHY_OUTPUT})"
            healthy_ok=true
        else
            echo "  /healthy check FAILED (Status: ${HTTP_STATUS_HEALTHY}, Unexpected Body: ${HEALTHY_OUTPUT})"
            healthy_ok=false # Ensure it's marked false if body check fails
        fi
    elif [[ "$HTTP_STATUS_HEALTHY" == "curl_error" ]]; then
         echo "  /healthy check FAILED (curl command error or timeout)"
         healthy_ok=false
    else
        echo "  /healthy check FAILED (HTTP Status: ${HTTP_STATUS_HEALTHY})"
        healthy_ok=false
    fi

    # Check /?validate_geo=healthy endpoint
    echo "  Checking ${VERIFY_URL_GEO}..."
    HTTP_STATUS_GEO=$(curl --connect-timeout 10 -s -L -o /dev/null -w "%{http_code}" "${VERIFY_URL_GEO}" || echo "curl_error")
     if [[ "$HTTP_STATUS_GEO" == "200" ]]; then
        GEO_OUTPUT=$(curl --connect-timeout 10 -s -L "${VERIFY_URL_GEO}" || echo "curl_error")
        if [[ "$GEO_OUTPUT" == "ok" ]]; then
            echo "  /?validate_geo=healthy check: OK (Status: ${HTTP_STATUS_GEO}, Body: ${GEO_OUTPUT})"
            geo_ok=true
        else
             echo "  /?validate_geo=healthy check FAILED (Status: ${HTTP_STATUS_GEO}, Unexpected Body: ${GEO_OUTPUT})"
             geo_ok=false
        fi
    elif [[ "$HTTP_STATUS_GEO" == "curl_error" ]]; then
         echo "  /?validate_geo=healthy check FAILED (curl command error or timeout)"
         geo_ok=false
    else
        echo "  /?validate_geo=healthy check FAILED (HTTP Status: ${HTTP_STATUS_GEO})"
        geo_ok=false
    fi

    # Check if both are okay
    if [[ "$healthy_ok" == true && "$geo_ok" == true ]]; then
        echo "Verification successful!"
        break # Exit the loop
    fi

    # Increment retries and wait
    retries=$((retries + 1))
    if [[ $retries -lt $MAX_RETRIES ]]; then
        echo "  Waiting ${WAIT_SECONDS} seconds before next check..."
        sleep ${WAIT_SECONDS}
    fi
done

# Final check after the loop
if [[ "$healthy_ok" != true || "$geo_ok" != true ]]; then
    echo "ERROR: Verification failed after ${MAX_RETRIES} attempts."
    echo "Please check the following:"
    echo "  - Load balancer configuration propagation (can take several minutes)."
    echo "  - Correctness of PROJECT_ID, DOMAIN_NAME, GTM_TAG_ID, GTM_TAG_PATH."
    echo "  - DNS for ${DOMAIN_NAME} points to the correct LB IP address."
    echo "  - SSL certificate for ${DOMAIN_NAME} is ACTIVE."
    echo "  - Manually test the URLs:"
    echo "    curl -v ${VERIFY_URL_HEALTHY}"
    echo "    curl -v ${VERIFY_URL_GEO}"
    # Optionally exit with an error code if needed for automation pipelines
    # exit 1
fi
echo "-----------------------------------------------------"

echo "--- GTM First-Party Load Balancer Configuration Finished ---"
echo "Review the verification output above."
echo "If checks succeeded, traffic to https://${DOMAIN_NAME}${GTM_TAG_PATH}/ should now be routed correctly."
echo "If checks failed, review the ERROR messages and troubleshooting steps."
echo "-----------------------------------------------------"

--- Starting GTM First-Party Load Balancer Configuration ---
Project ID:                 cloud-for-marketing-demo
Domain Name:                cloud-marketing-demos.com
GTM Tag ID:                 G-DS0F7HKZTC
GTM Tag Path:               /metrics
Load Balancer URL Map:      metrics-lb-url-map-https
Default Backend Service:    metrics-lb-backend-service
GTM FPS Domain:             G-DS0F7HKZTC.fps.goog
GTM NEG Name:               metrics-lb-gtm-fps-neg
GTM Backend Service Name:   metrics-lb-gtm-fps-backend-service
GTM Path Matcher Name:      metrics-lb-gtm-path-matcher
Verification Retries:       15
Verification Wait Seconds:  30
-----------------------------------------------------
Creating/Verifying Global Internet NEG: metrics-lb-gtm-fps-neg...
Created [https://www.googleapis.com/compute/v1/projects/cloud-for-marketing-demo/global/networkEndpointGroups/metrics-lb-gtm-fps-neg].
Created network endpoint group [metrics-lb-gtm-fps-neg].
Adding/Verifying endpoint G-DS0F7HKZTC.fps.goog:443 

### 🗑️ Uninstall GTM Load Balancer Integration Components
Uncomment the cell below to run the uninstall steps.

In [145]:
# # @markdown Run this cell to remove the GTM-specific components (NEG, Backend Service) and revert the URL map changes made by the GTM setup script.

# # @markdown This script *only* removes the GTM integration parts. It does **not** delete the main Load Balancer components (IP, SSL cert, main backend, forwarding rules, etc.). Run a separate full LB uninstall script if you need to remove everything.

# # @markdown ---
# # @markdown **Ensure the Python configuration cell above has been run with the correct settings corresponding to the resources you want to delete.**


# %%shell
# #!/bin/bash

# # Allow script to continue even if a command fails, reporting errors
# set +e # Important for cleanup scripts

# # --- Configuration ---
# # Reads configuration from environment variables set by the Python cell.
# # Ensure the Python config cell was run correctly before executing this.

# echo "--- Starting GTM First-Party Load Balancer Component DELETION ---"
# echo "Project ID:                 ${PROJECT_ID}"
# echo "Main URL Map:             ${GCLOUD_URL_MAP}"
# echo "Default Backend Service:    ${GCLOUD_BACKEND_SERVICE}" # Used for reverting URL map
# echo "GTM Backend Service Name:   ${GTM_BACKEND_SERVICE_NAME}" # To be deleted
# echo "GTM NEG Name:               ${GTM_NEG_NAME}" # To be deleted
# echo "GTM Path Matcher Name:      ${GTM_PATH_MATCHER_NAME}" # Implicitly removed by URL map revert
# echo "-----------------------------------------------------"
# echo "NOTE: Errors for 'Not Found' are expected if resources were already deleted or never created."
# echo "This script attempts to revert the URL Map and delete the GTM-specific backend service and NEG."
# echo "-----------------------------------------------------"


# # 1. Revert URL Map to remove GTM routing rules
# # We achieve this by re-importing the URL map with only the default service defined.
# echo "Reverting URL Map '${GCLOUD_URL_MAP}' to only use default service '${GCLOUD_BACKEND_SERVICE}'..."

# # Get the full URL of the default backend service needed for the YAML
# echo "Fetching URL for default backend service: ${GCLOUD_BACKEND_SERVICE}..."
# DEFAULT_SERVICE_URL=$(gcloud compute backend-services describe "${GCLOUD_BACKEND_SERVICE}" --project="${PROJECT_ID}" --global --format="value(selfLink)" 2>/dev/null)

# if [ -z "$DEFAULT_SERVICE_URL" ]; then
#   echo "ERROR: Could not retrieve the URL for the default backend service '${GCLOUD_BACKEND_SERVICE}'. Cannot revert URL map. Please ensure it exists."
#   # Decide if you want to exit or continue trying to delete other resources
#   # exit 1 # Or comment out to let it try deleting the backend/NEG
# else
#   echo "Default backend service URL: ${DEFAULT_SERVICE_URL}"
#   # Define the minimal YAML configuration for reverting the URL Map
#   URL_MAP_REVERT_YAML_FILE="url_map_revert.yaml"
#   cat << EOF > ${URL_MAP_REVERT_YAML_FILE}
# # Revert configuration for URL Map: ${GCLOUD_URL_MAP}
# # Sets only the default service.
# kind: compute#urlMap
# name: ${GCLOUD_URL_MAP}
# defaultService: ${DEFAULT_SERVICE_URL}
# # No hostRules or pathMatchers defined here intentionally
# EOF

#   echo "Generated Revert YAML:"
#   cat ${URL_MAP_REVERT_YAML_FILE}
#   echo "---"

#   # Import the minimal configuration, overwriting the existing URL Map
#   gcloud compute url-maps import "${GCLOUD_URL_MAP}" \
#       --project="${PROJECT_ID}" \
#       --source="${URL_MAP_REVERT_YAML_FILE}" \
#       --global \
#       --quiet || echo "WARNING: Failed to import revert configuration for URL Map '${GCLOUD_URL_MAP}'. It might not exist or there was an issue."

#   # Clean up the temporary YAML file
#   rm -f ${URL_MAP_REVERT_YAML_FILE}
# fi
# echo "URL Map revert step complete."
# echo "-----------------------------------------------------"


# # 2. Delete Backend Service for GTM FPS
# # First, try detaching the NEG if the backend service still exists
# echo "Attempting removal of NEG '${GTM_NEG_NAME}' from GTM Backend Service '${GTM_BACKEND_SERVICE_NAME}' (if backend exists)..."
# if gcloud compute backend-services describe "${GTM_BACKEND_SERVICE_NAME}" --project="${PROJECT_ID}" --global &> /dev/null; then
#     # Backend exists, try removing the NEG
#     gcloud compute backend-services remove-backend "${GTM_BACKEND_SERVICE_NAME}" \
#         --project="${PROJECT_ID}" \
#         --network-endpoint-group="${GTM_NEG_NAME}" \
#         --global-network-endpoint-group \
#         --global \
#         --quiet || echo "WARNING: Failed to remove NEG '${GTM_NEG_NAME}' from GTM backend service '${GTM_BACKEND_SERVICE_NAME}' (may have already been removed or GTM NEG doesn't exist)."

#     # Now delete the backend service itself
#     echo "Deleting GTM Backend Service: ${GTM_BACKEND_SERVICE_NAME}..."
#     gcloud compute backend-services delete "${GTM_BACKEND_SERVICE_NAME}" \
#         --project="${PROJECT_ID}" \
#         --global \
#         --quiet || echo "WARNING: Failed to delete GTM Backend Service '${GTM_BACKEND_SERVICE_NAME}' (check dependencies or permissions)."
# else
#     echo "INFO: GTM Backend service '${GTM_BACKEND_SERVICE_NAME}' not found, skipping NEG removal and backend deletion."
# fi
# echo "GTM Backend Service deletion step complete."
# echo "-----------------------------------------------------"


# # 3. Delete Internet Network Endpoint Group (NEG) for GTM FPS
# echo "Deleting Global Internet NEG: ${GTM_NEG_NAME}..."
# gcloud compute network-endpoint-groups delete "${GTM_NEG_NAME}" \
#     --project="${PROJECT_ID}" \
#     --global \
#     --quiet || echo "WARNING: Failed to delete GTM Internet NEG '${GTM_NEG_NAME}' (may not exist or was already deleted)."
# echo "GTM NEG deletion step complete."
# echo "-----------------------------------------------------"

# echo "--- GTM First-Party Load Balancer Component Deletion Finished ---"
# echo "Review output above for any unexpected WARNING messages."
# echo "The URL Map '${GCLOUD_URL_MAP}' should now only route to the default backend '${GCLOUD_BACKEND_SERVICE}'."
# echo "The GTM-specific backend service ('${GTM_BACKEND_SERVICE_NAME}') and NEG ('${GTM_NEG_NAME}') should be deleted."
# echo "-----------------------------------------------------"


--- Starting GTM First-Party Load Balancer Component DELETION ---
Project ID:                 cloud-for-marketing-demo
Main URL Map:             metrics-lb-url-map-https
Default Backend Service:    metrics-lb-backend-service
GTM Backend Service Name:   metrics-lb-gtm-fps-backend-service
GTM NEG Name:               metrics-lb-gtm-fps-neg
GTM Path Matcher Name:      metrics-lb-gtm-path-matcher
-----------------------------------------------------
NOTE: Errors for 'Not Found' are expected if resources were already deleted or never created.
This script attempts to revert the URL Map and delete the GTM-specific backend service and NEG.
-----------------------------------------------------
Reverting URL Map 'metrics-lb-url-map-https' to only use default service 'metrics-lb-backend-service'...
Fetching URL for default backend service: metrics-lb-backend-service...
Default backend service URL: https://www.googleapis.com/compute/v1/projects/cloud-for-marketing-demo/global/backendServices/metric

## Finalize GTM Containers
Add your domain to the container tags and configs.

* After the infrastructure is set up, you need to **manually update your GTM Server container configuration** within the Google Tag Manager UI to add your custom domain (`cloud-marketing-demos.com` in this example) as a valid serving URL.